# Transfer Learning Example

### Problem Statement

In this tutorial, we shall improve on the exisiting classifier but taking the Transfer Learning approach.

We shall use the <b>VGG16 CNN Architecture from Keras Framework pre-trained on ImageNet Dataset</b>.

We shall follow the similar steps as before with a few changes in defining the model:<br>
    1. Load the Data CIFAR10<br>
    2. Load the pre-trained VGG16 model from Keras and Freeze the first few layers<br>
    3. Add some end layers to cater for Classifying 10 Classes as in CIFAR10<br>
    4. Train the model<br>
    5. Evaluate the model on Test set<br>
 

In [ ]:
import tensorflow as tf
import keras
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
%matplotlib inline
tf.logging.set_verbosity(tf.logging.ERROR)

# Load the Data

In [ ]:
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator

# Load Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Mention the Class Name List
class_list = ['airplane','automobile','bird','cat','deer','dog',
              'frog','horse','ship','truck']

# # Convert a one-hot vector for the test-labels
y_train = keras.utils.to_categorical(y_train, num_classes=len(class_list))


# # Split the test set to Validation & Test set
num_test_samples = x_test.shape[0]
x_val, y_val = x_test[0:num_test_samples//2,:,:], y_test[0:num_test_samples//2]
x_test, y_test = x_test[num_test_samples//2:,:,:], y_test[num_test_samples//2:]

y_val = keras.utils.to_categorical(y_val, num_classes=len(class_list))

# Create DATA GENERARTOR with data augmentation
train_data_gen = ImageDataGenerator(rescale=1.0/255,horizontal_flip=False)
train_data_gen.fit(x_train)
train_generator = train_data_gen.flow(x_train, y_train, batch_size=32)

val_data_gen = ImageDataGenerator(rescale=1.0/255,horizontal_flip=False)
val_data_gen.fit(x_val)
val_generator = val_data_gen.flow(x_val, y_val, batch_size=32)



# Load the Model

The pre-trained model we are using here is the VGG16 model which has a total of 16 layers including the fully-connected layer. This model is pre-trained on the huge ImageNet Dataset (over 14M Images)

We are going to Freeze the layers until layer `block3_pool` and add few layers of our own to retrain on the CIFAR10 training set. 

For using other base models - [Check this page](https://keras.io/applications/). Base Models can be swapped with any of the ones present on the link.

In [ ]:
from keras.models import Model
from keras.applications import vgg16 as vgg
from keras.layers import Flatten, Dense, BatchNormalization, GlobalAveragePooling2D
from keras.optimizers import Adam, SGD
# Get the pre-trained model
base_model = vgg.VGG16(weights='imagenet',
                          include_top=False,
                          input_shape=(32,32,3))

# # Visualize the base_model and check the layer name to 'chop at'
# base_model.summary()

# Get the Last output from the base model from last layer
last_output = base_model.get_layer('block3_pool').output

# Add new Layers
                                        # globalavg
                                        # bn
                                        # Dense + Relu
                                        # Dense + Relu
                                        # Dense + Softmax

# Build the model
model = Model(inputs=base_model.input, outputs=pred)

# Freeze layers of Base Model from getting Trained again. 
# The weights in these layers are not affected by BackPropagation
for layer in base_model.layers:
    layer.trainable = False
        
# Compile the Model with the Loss function, Optimizer and Accuracy Metric
model.compile(loss='categorical_crossentropy',optimizer=Adam(),metrics=['accuracy'])

# Visualize the model
model.summary()

# Train the Model using Generator

In [ ]:
# Train the model using the Image Generator
model.fit_generator(train_generator,
                   steps_per_epoch=50000//32,
                   validation_data=val_generator,
                   validation_steps=5000//32,
                   epochs=10,
                   verbose=1)

# Test the Model

In [ ]:
# Predict on Test Set
y_pred = np.argmax(model.predict(x_test), axis=-1).reshape(5000,1)   #^

acc = np.sum(y_test == y_pred) / y_test.shape[0]
# Print Accuracy
print("Test Accuracy :{}".format(acc*100))

# Visualize Predictions



In [ ]:

import pdb
# UTILS FOR VISUALIZATION
def plot_predictions(model,dataset,
                    dataset_labels,label_dict,
                    batch_size,grid_height=4,grid_width=4):
    if model:
        f, ax = plt.subplots(grid_width, grid_height)
        f.set_size_inches(12, 12)
       
        random_batch_indx = np.random.permutation(np.arange(0,len(dataset)))[:batch_size]
       
        img_idx = 0
        for i in range(0, grid_width):
            for j in range(0, grid_height):
                actual_label = label_dict.get(dataset_labels[random_batch_indx[img_idx]].argmax())
                       
                # make prediction and find score
                prediction = model.predict(dataset[random_batch_indx[img_idx]].reshape(1,32,32,3))[0]
                preds = [label_dict[idx] for idx in np.argsort(prediction)[::-1][:1]]
                confs_ = np.sort(prediction)[::-1][:1]
                ax[i][j].axis('off')
                ax[i][j].set_title('Actual:'+actual_label+\
                                    '\nPredicted:'+preds[0] + \
                                    '(' +str(round(confs_[0],2)) + ')')
                ax[i][j].imshow(dataset[random_batch_indx[img_idx]])
                img_idx += 1

        plt.subplots_adjust(left=0, bottom=0, right=1,top=1, wspace=0.4, hspace=0.55)

In [ ]:

# VISUALIZE ON A BATCH SIZE OF IMAGES
plot_predictions(model=model,
                 dataset=(x_test/255.),
                 dataset_labels=keras.utils.to_categorical(y_test,num_classes=10),
                 label_dict={class_list.index(i) : i for i in class_list},
                  batch_size=16)